# MusicXML Visualization

This notebook is phase two of this project, attempting to take what I've learned
about gathering data and visualize it.

## Prep

### Imports

In [1]:
# imports and constants
import music21
from pprint import pprint
from difficulty import *
from type21 import *
import warnings
import os
import numpy as np
import pandas as pd

### Parse MusicXML File

In [2]:
score = music21.converter.parse("../music/xml/musicalion/beethoven_egmont.musicxml")

## Extraction and Organization

In [3]:
currTempo = None
score_list = []

for part in score:
    try:
        instrument = part.getInstrument().instrumentName
    except:
        print(part)
        continue
    record_in_data_frame = True
    for element in part.recurse():
        # define items to be added to the data frame
        note_start = None
        note_end = None
        pitch = None
        volume = None
        tempo = None
        note_type = None

        if isChord(element):
            for note in element:
                note_start = note.offset
                note_end = note.offset + note.quarterLength
                pitch = note.pitch.ps
                volume = note.volume.realized
                tempo = currTempo
                note_type = type(note)
        elif isNote(element):
            note_start = element.offset
            note_end = element.offset + element.quarterLength
            pitch = element.pitch.ps
            volume = element.volume.realized
            note_type = type(element)
        elif type(element) == music21.tempo.MetronomeMark:
            record_in_data_frame = False
            currTempo = element.number
        elif isRest(element):
            note_start = element.offset
            note_end = element.offset + element.quarterLength
            note_type = type(element)
        if record_in_data_frame:
            score_list.append([note_start, note_end, pitch, volume, currTempo, note_type, instrument])


<music21.text.TextBox 'Seite #'>
<music21.layout.ScoreLayout>
<music21.layout.StaffGroup <music21.stream.PartStaff P1-Staff1><music21.stream.PartStaff P1-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P5-Staff1><music21.stream.PartStaff P5-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P8-Staff1><music21.stream.PartStaff P8-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P10-Staff1><music21.stream.PartStaff P10-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P1-Staff1><music21.stream.PartStaff P1-Staff2><music21.stream.Part 2 Oboi><music21.stream.Part 2 Clarinetti in B><music21.stream.Part 2 Fagotti>>
<music21.layout.StaffGroup <music21.stream.PartStaff P5-Staff1><music21.stream.PartStaff P5-Staff2><music21.stream.Part 2 Trombe in F><music21.stream.Part TImpani in F, c>>
<music21.layout.StaffGroup <music21.stream.PartStaff P8-Staff1><music21.stream.PartStaff P8-Staff2><music21.stream.Part Viola><music21.stream.PartStaff P10-Staf

In [5]:
df = pd.DataFrame(score_list, columns=['note_start', 'note_end', 'pitch', 'volume', 'tempo', 'note_type', 'instrument'])
df

,note_start,note_end,pitch,volume,tempo,note_type,instrument
0,None,None,NaN,NaN,None,None,Midi_74
1,None,None,NaN,NaN,None,None,Midi_74
2,None,None,NaN,NaN,None,None,Midi_74
3,None,None,NaN,NaN,None,None,Midi_74
4,None,None,NaN,NaN,None,None,Midi_74
...,...,...,...,...,...,...,...
25390,None,None,NaN,NaN,None,None,Midi_43
25391,0.0,1.0,41.0,1.0,None,<class 'music21.note.Note'>,Midi_43
25392,1.0,2.0,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_43
25393,2.0,4.0,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_43


In [ ]:
df1 = df[df['instrument'] == 'Midi_71']
df1.plot(x="note_start", figsize=(20, 20), subplots=True)